# Tutorial: Quantum Espresso Wrapper

In this tutorial we will go through a simple example of how to use the wrapper for the Quantum Espresso simulation engine.
You can find the wrapper [here](https://github.com/simphony/quantum-espresso-wrapper).

## Background
This is an example of a slightly different design based upon the input-output functionality of certain simulation engines such as Quantum Espresso and Gromacs.


## Let's get hands-on
### Installation
To run the local installation of Quantum Espresso, simply run `./install_engine.sh`. This should check for the prerequisites and compile the code for Quantum Espresso for you.

If the script runs into an error finding openmpi-bin or something like that, try running `apt-get update` and try again. 
Once the installation has completed, try running `pw.x` to see if the installation has succeeded. If this does not work, then try adding `export PATH=$PATH:/home/username/qe-6.1/bin/` at the end of `.bashrc` located at your home folder.  
  
Once you have verified that `pw.x` works, install the ontology via `pico install ontology.simlammps.yml`, and make sure to run `python3 setup.py` located in the root of the quantum espresso wrapper folder.   

That should be all needed to use Quantum Espresso!

### Simple example

This is an adaptation of quantum-espresso-wrapper/examples/Simple.py. As usual, we need to import the necessary components:

In [1]:
import numpy as np 

from osp.core.namespaces import QE
from osp.core.utils import pretty_print
from osp.wrappers.quantumespresso.qe_session import qeSession

Next, we create simulation and its K points, which determine at what points it samples the cell

In [2]:
sim = QE.Simulation()
k = QE.K_POINTS(vector = (7, 7, 7), unit = "")


Next, we create a cell, the element Silicon, a pseudopotential, an atom and the cell parameters. Note that the pseudopotential files should ALWAYS be located inside of a folder named `$PSEUDO_DIR` inside of wherever you are running the simulation.

In [3]:
SiCell = QE.Cell()
Si = QE.Element(name = "Si")
SiPseudo = QE.PSEUDOPOTENTIAL(name = "Si.pbe-n-kjpaw_psl.1.0.0.UPF")
Si1 = QE.Atom()
SiParams = QE.CellParams()
celldm1 = QE.Celldm1(value = 5.43070, unit = "au")

Next, we connect these all to each other using the `add` method.

In [4]:
Si.add(SiPseudo, Si1)
Si.add(QE.Mass(value = 28.085, unit = "amu"))
SiCell.add(Si1, SiParams)
Si1.add(QE.Position(vector = (0, 0, 0), unit = ""))
SiCell.add(celldm1)

<qe.Celldm1: be8f3915-3eb7-4221-a441-345eda51832b,  CoreSession: @0x7feae9717370>

We specify the cell parameters:

In [5]:
SiParams.add(QE.CellParameterX(vector = (0.5, 0.5, 0), unit = ""),
             QE.CellParameterY(vector = (0.5, 0, 0.5), unit = ""),
             QE.CellParameterZ(vector = (0, 0.5, 0.5), unit = ""))

[<qe.CellParameterX: 0ebdeed9-1d8a-498d-94c9-bafccb05d652,  CoreSession: @0x7feae9717370>,
 <qe.CellParameterY: fea8789c-8c07-49f9-9971-8d42bdd6ba3f,  CoreSession: @0x7feae9717370>,
 <qe.CellParameterZ: 1474d106-4204-428d-827b-2d5e2cb4af51,  CoreSession: @0x7feae9717370>]

And then we add everything created so far to the simulation:


In [6]:
sim.add(SiCell)
sim.add(Si)
sim.add(k)

<qe.K_POINTS: 6847a5f0-8d20-4f73-9eb3-043e78053182,  CoreSession: @0x7feae9717370>

While we're add it, let's add some variables to the simulation which we can check to see if they have been updated. They will not be taken into account when simulating, so they're there for control purposes.


In [7]:
sim.add(QE.Pressure(value = 100, unit = "kbar"))
sim.add(QE.StressTensor(tensor2 = np.zeros((3, 3)), unit = "kbar"))

<qe.StressTensor: 2f302f8b-89b8-4d7a-a2a7-4f6e19737f00,  CoreSession: @0x7feae9717370>

Let's check out what this simulation looks like now with the `pretty_print` function:

In [8]:
pretty_print(sim)

- Cuds object:
  uuid: 903145ad-50e3-46fc-9d28-1aa1ec364e8a
  type: qe.Simulation
  superclasses: cuba.Class, cuba.Entity, qe.Simulation
  description: 
    All components of the simulation that are needed to run the model

   |_Relationship qe.HAS_PART:
     -  qe.Cell cuds object:
     .  uuid: 67a0fcb4-4977-49df-9bcb-13bca17b2763
     .   |_Relationship qe.HAS_PART:
     .     -  qe.Atom cuds object:
     .     .  uuid: 1bad1c25-609a-4bc0-8c65-3c0167cfdbe2
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.Position cuds object:
     .     .        uuid: c70afbb3-0012-488d-8059-b44d775c6b23
     .     .        vector: [0. 0. 0.]
     .     .        unit: 
     .     -  qe.CellParams cuds object:
     .     .  uuid: f444899a-e850-4ab2-b79e-c91026523eb3
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.CellParameterX cuds object:
     .     .     .  uuid: 0ebdeed9-1d8a-498d-94c9-bafccb05d652
     .     .     .  vector: [0.5 0.5 0. ]
     .     .     .  un

Now, it's time to get the simulation running:

In [12]:
session = qeSession()
quantum_espresso_wrapper = QE.QEWrapper(session = session)
quantum_espresso_wrapper.add(sim)
print("Running calculation...")

quantum_espresso_wrapper.session._run(simulation = sim, prefix = "si", command_type = "pw.x", calculation_type = "scf", root = "", CONTROL = {'pseudo_dir': "'.'"})

Running calculation...
/mnt/c/iwm/docs/si.pwscf.in
pw.x -i /mnt/c/iwm/docs/si.pwscf.in > /mnt/c/iwm/docs/si.pwscf.out


Now let's check the results of our calculation:

In [10]:
pretty_print(sim)

- Cuds object:
  uuid: 903145ad-50e3-46fc-9d28-1aa1ec364e8a
  type: qe.Simulation
  superclasses: cuba.Class, cuba.Entity, qe.Simulation
  description: 
    All components of the simulation that are needed to run the model

   |_Relationship qe.HAS_PART:
     -  qe.Cell cuds object:
     .  uuid: 67a0fcb4-4977-49df-9bcb-13bca17b2763
     .   |_Relationship qe.HAS_PART:
     .     -  qe.Atom cuds object:
     .     .  uuid: 1bad1c25-609a-4bc0-8c65-3c0167cfdbe2
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.Position cuds object:
     .     .        uuid: c70afbb3-0012-488d-8059-b44d775c6b23
     .     .        vector: [0. 0. 0.]
     .     .        unit: 
     .     -  qe.CellParams cuds object:
     .     .  uuid: f444899a-e850-4ab2-b79e-c91026523eb3
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.CellParameterX cuds object:
     .     .     .  uuid: 0ebdeed9-1d8a-498d-94c9-bafccb05d652
     .     .     .  vector: [0.5 0.5 0. ]
     .     .     .  un

As you can see, the original part of the cuds tree is still there, with everything mostly the same. The new parts are:

- The qe.PwOut cuds object. This is the output file of the simulation, in case there is something that the wrapper does not parse but that you would still like to see.
- The qe.TotalEnergy cuds object. This was parsed from the qe.PwOut file itself.
- The qe.Force cuds object. This represents the force exerted on the atom(s).

The updated parts are:

- The qe.Pressure cuds object, having changed in value from 100 kbar to 5723.64 kbar.
- The qe.StressTensor cuds object, which is no longer zero.

Let's see if we can do better and calculate some bands structures:

In [21]:
quantum_espresso_wrapper.session._run(prefix = "si", command_type = "pw.x", calculation_type = "bands")
quantum_espresso_wrapper.session._run(prefix = "si", command_type = "bands.x", calculation_type = "")

TypeError: _run() missing 1 required positional argument: 'simulation'

Although the cuds structure won't have changed much by this, the data is there in the folder.

Now let's try to relax this cell. While it isn't a real cell, we can still perform the calculations to relax it to know what the movement of the atoms would be like if it were a real cell (warning, perform vc-relax type calculations with caution. These examples are designed to be lightweight and non-indicative of real-world applications).

In [22]:
quantum_espresso_wrapper.session._run(simulation = sim, prefix = "si", command_type = "pw.x", calculation_type = "relax", IONS = {'ion_dynamics': "'bfgs'"})

TypeError: _run() missing 1 required positional argument: 'simulation'

In [23]:
pretty_print(sim)

- Cuds object:
  uuid: 90361daa-6905-4566-979e-11b3b0dd4e85
  type: qe.Simulation
  superclasses: cuba.Class, cuba.Entity, qe.Simulation
  description: 
    All components of the simulation that are needed to run the model

   |_Relationship qe.HAS_PART:
     -  qe.Cell cuds object:
     .  uuid: f7548873-28e9-4d76-86da-6fddb687d29e
     .   |_Relationship qe.HAS_PART:
     .     -  qe.Atom cuds object:
     .     .  uuid: 53114a1f-ebbb-4e4b-a115-080925d9eaa8
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.Position cuds object:
     .     .        uuid: 48d4483b-7c72-4454-8041-581dc73fd216
     .     .        vector: [0. 0. 0.]
     .     .        unit: 
     .     -  qe.CellParams cuds object:
     .     .  uuid: 9d61e990-2509-474b-935e-618ca11bb40d
     .     .   |_Relationship qe.HAS_PART:
     .     .     -  qe.CellParameterX cuds object:
     .     .     .  uuid: f2655054-efa7-4b39-9f0a-cf6453be68ec
     .     .     .  vector: [0.5 0.5 0. ]
     .     .     .  un

In this example, the position hasn't changed. So let's spice things up a little bit and add another atom, and then relax:

In [24]:
Si2 = QE.Atom()
Si2.add(QE.Position(vector = (0.25, 0.25, 0.26), unit = ""))
SiCell.add(Si)
Si.add(Si2)
pretty_print(sim)

ValueError: <qe.Element: 14dacecb-023c-4ace-9e83-35b0ecaa1032,  CoreSession: @0x7f3d2d38bfa0> is already in the container

In [25]:
quantum_espresso_wrapper.session._run(simulation = sim, prefix = "si", command_type = "pw.x", calculation_type = "relax", IONS = {'ion_dynamics': "'bfgs'"})
pretty_print(sim)

TypeError: _run() missing 1 required positional argument: 'simulation'